In [106]:
import pandas as pd
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource
from stock_keys import api_key
from stock_saver import Stock
import stock_chart as sc
import math

In [163]:
ticker = 'AAPL'
intraday = True
daily = None
st = Stock(ticker, api_key, intraday, daily)

[AAPL] Intraday : 2019-05-28 15:12:00
[AAPL] 407 values added.


In [167]:
def compute_spread(df):
    df['spread'] = df.close - df.open

def compute_simple_ma(df, t=200):
    # Cross-section of multi-timeframe MA is used as an exit strategy.
    # AlphaVantage offers MA over 60 or 200 points at any timeframe.
    df['simple_ma'] = df.close.rolling(window=(200)).mean().fillna(0)
    
def draw_simple_ma(p, df):
    p.line('date', 'simple_ma', source = ColumnDataSource(df), line_color="black")
    return(p)
    
def exponential_ma(values, t=200):
    # AlphaVantage uses 60 or 200 data points for each EMA.
    k = 2 / (t + 1)
    exp_ma = values[0]
    for v in values[1:]:
        exp_ma = k * v + (1 - k) * exp_ma
    return(exp_ma)

def compute_exponential_ma(df, t=200):
    df['exponential_ma'] = df.close.rolling(window=(t)).apply(exponential_ma, args=(t,)).fillna(0)

def draw_exponential_ma(p, df):
    p.line('date', 'exponential_ma', source = ColumnDataSource(df), line_color="black")
    return(p)

def compute_bollinger_bands(df, t=20, f=2):
    # Here, t is window size and f is standard deviation which is usually 2 but could be increased for safety.
    df['tp'] = (df.high + df.low + df.close) / 3
    mid_band = []
    upper_band = []
    lower_band = []
    l = len(df.index)
    for i in range(t):
        mid_band.append(0)
        upper_band.append(0)
        lower_band.append(0)
    c = 0
    while (c + t) < l:
        # Here, the mid is calculated as Simple MA over tp.
        mid = df.iloc[c + t].tp
        mid_band.append(mid)
        upper_band.append(mid + f * df.iloc[c : (c + t)].tp.std())
        lower_band.append(mid - f * df.iloc[c : (c + t)].tp.std())
        c += 1
    print(len(mid_band), len(upper_band), len(lower_band))
    df['mid_band'] = mid_band
    df['upper_band'] = upper_band
    df['lower_band'] = lower_band    

def draw_bollinger_bands(p, df):
    p.line('date', 'upper_band', source = ColumnDataSource(df), line_color="red")
    p.line('date', 'mid_band', source = ColumnDataSource(df), line_color="blue")
    p.line('date', 'lower_band', source = ColumnDataSource(df), line_color="green")
    return(p)

In [172]:
day = 28
month = 5
year = 2019
df = pd.DataFrame(data=st.df)
compute_spread(df)
compute_simple_ma(df, 200)
compute_exponential_ma(df, 200)
compute_bollinger_bands(df, 20, 3)
df = df[df.index.map(lambda x: (df.date[x].year) == year and (df.date[x].month) == month and (df.date[x].day) == day)]
output_notebook()

/home/demq/Documents/.anaconda3/envs/dq/lib/python3.7/site-packages/ipykernel_launcher.py:22: FutureWarning: Currently, 'apply' passes the values as ndarrays to the applied function. In the future, this will change to passing it as Series objects. You need to specify 'raw=True' to keep the current behaviour, and you can pass 'raw=False' to silence this warning


2604 2604 2604


Loading BokehJS ...

In [173]:
p = figure(x_axis_type="datetime", plot_width=1000, plot_height=350, x_range = (df.date.min(), df.date.max()), y_range = (df.low.min(), df.high.max()))
p = sc.set_title(p, st.ticker)
p = sc.draw_price(p, df)
p = sc.draw_volume(p, df, False)
# p = draw_simple_ma(p, df)
# p = draw_exponential_ma(p, df)
p = draw_bollinger_bands(p, df)

show(p)